In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

2023-07-13 11:01:33.276205: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 11:01:33.316409: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 11:01:33.317549: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 11:01:33.952325: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Define the Q-Network model
def create_q_network(input_shape, action_space):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(action_space, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

In [3]:
class ENV():
    def __init__(self):
        self.env_col = 11
        self.env_row = 11
        self.state = (0, 0)
        self.action_space = 4
        self.reward = 0
        self.done = False
        self.goal = (3, 3)

    def reset(self):
        self.state = (0, 0)
        self.reward = 0
        self.done = False
        return self.state
    
    def step(self, action):
        if action == 0: # up
            self.state = (self.state[0], self.state[1] + 1)
        elif action == 1: # down
            self.state = (self.state[0], self.state[1] - 1)
        elif action == 2: # left
            self.state = (self.state[0] - 1, self.state[1])
        elif action == 3: # right
            self.state = (self.state[0] + 1, self.state[1])
        else:
            raise ValueError("Invalid action")
        if self.state == self.goal:
            self.reward = 100
            self.done = True
        elif self.state[0] < 0 or self.state[0] >= self.env_col or self.state[1] < 0 or self.state[1] >= self.env_row:
            self.reward = -100
            self.done = True
        else:
            self.reward = -1
        return self.state, self.reward, self.done


In [4]:
#Define the DQN agent
class DQNAgent:
    def __init__(self, state_shape, action_space):
        self.state_shape = state_shape
        self.action_space = action_space
        self.memory = []
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_decay = 0.995  # Decay rate for exploration rate
        self.epsilon_min = 0.01  # Minimum exploration rate
        self.model = create_q_network(state_shape, action_space)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.action_space)
        print("state in action: ",state)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        # Debug
        # print(self.memory)
        # batch = np.random.choice(self.memory, batch_size, replace=False)
        batch_ch = np.random.choice(len(self.memory), batch_size, replace=False)
        batch = [self.memory[i] for i in batch_ch]
        for state, action, reward, next_state, done in batch:
            next_state = np.reshape(state, self.state_shape)
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [5]:
# Initialize the environment and agent
state_shape = (2,)  # Example state shape, adjust according to your actual state representation # Only x and y coordinates
action_space = 4  # Example action space size, adjust according to your actual actions # 4 actions: up, down, left, right
agent = DQNAgent(state_shape, action_space)
env = ENV()

2023-07-13 11:01:34.651669: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2023-07-13 11:01:34.651724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: pradeep-Victus-by-HP-Laptop-16-e0xxx
2023-07-13 11:01:34.651730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: pradeep-Victus-by-HP-Laptop-16-e0xxx
2023-07-13 11:01:34.651793: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.54.3
2023-07-13 11:01:34.651816: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  530.41.03  Release Build  (dvs-builder@U1

In [6]:
#Training loop
num_episodes = 1000  # Set the number of training episodes
batch_size = 32  # Set the batch size for replay
for episode in range(num_episodes):
    state = env.reset()
    state = np.reshape(state, state_shape)
    done = False
    while not done:
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, state_shape)
        agent.remember(state, action, reward, next_state, done)
        state = next_state

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

[(array([0, 0]), 1, -100, array([ 0, -1]), True), (array([0, 0]), 1, -100, array([ 0, -1]), True), (array([0, 0]), 2, -100, array([-1,  0]), True), (array([0, 0]), 1, -100, array([ 0, -1]), True), (array([0, 0]), 3, -1, array([1, 0]), False), (array([1, 0]), 2, -1, array([0, 0]), False), (array([0, 0]), 2, -100, array([-1,  0]), True), (array([0, 0]), 1, -100, array([ 0, -1]), True), (array([0, 0]), 1, -100, array([ 0, -1]), True), (array([0, 0]), 2, -100, array([-1,  0]), True), (array([0, 0]), 2, -100, array([-1,  0]), True), (array([0, 0]), 1, -100, array([ 0, -1]), True), (array([0, 0]), 2, -100, array([-1,  0]), True), (array([0, 0]), 2, -100, array([-1,  0]), True), (array([0, 0]), 0, -1, array([0, 1]), False), (array([0, 1]), 3, -1, array([1, 1]), False), (array([1, 1]), 1, -1, array([1, 0]), False), (array([1, 0]), 0, -1, array([1, 1]), False), (array([1, 1]), 1, -1, array([1, 0]), False), (array([1, 0]), 2, -1, array([0, 0]), False), (array([0, 0]), 2, -100, array([-1,  0]), T

ValueError: in user code:

    File "/home/pradeep/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/home/pradeep/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/pradeep/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/home/pradeep/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/home/pradeep/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/pradeep/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 253, in assert_input_compatibility
        ra

In [ ]:
# Use the trained agent to control the agent in the environment
state = env.reset()
state = np.reshape(state, state_shape)
done = False
while not done:
    action = agent.act(state)
    next_state, _, done = env.step(action)
    next_state = np.reshape(next_state, state_shape)
    # Take action with the agent in the environment
    state = next_state
    print(state)